In [ ]:
from IPython.core.display import HTML
with open('../style.css') as f:
    css = f.read()
HTML(css)

# Tic-Tac-Toe via Bitboards

This notebook defines the game [tic-tac-toe](https://en.wikipedia.org/wiki/Tic-tac-toe).
It is played on a $3 \times 3$ board.  There are two players, which are called `X`and `O`.  Player `X` starts.  Player `X` always puts an `'X'` into an empty field on the board, while player `O` always puts an `'O'` in an empty field of the board.  The goal of the game for player `X` is to get three **Xs** into a row, column, or diagonal line, while player `O` needs to get three **Os** into a row, column, or diagonal line.

In this notebook, the board is represented as a *bitboard*, i.e. the states are represented as integers.  If `s` is a state, the first nine bits of `s` specify the positions of the `X`es, while the second nine bits specify the positions of the `O`s.  Player `X` is encoded as the number `0`,  while player `O` is encoded as the number `1`.

In [ ]:
Players = [0, 1]

States are represented as integers.  The first 9 bits encode the marks of the player `X`, the next 9 bits encode the marks of the player `O`.  Initially, the board is empty and hence no bit is set.

In [ ]:
Start = 0

The function `set_bits` takes one parameter:
* `Bits` is a list of numbers that specify bit positions in an integer.

The function returns an integer such that the bits specified in `Bits` are set. Counting starts with $0$.

In [ ]:
def set_bits(Bits):
    result = 0
    for b in Bits:
        result |= 1 << b
    return result

In [ ]:
assert set_bits([0, 1, 4]) == 2**0 + 2**1 + 2**4

The function `set_bit(n)` returns a number that has exactly the $\texttt{n}^\textrm{th}$ bit set.

In [ ]:
set_bit = lambda n: 1 << n

In [ ]:
assert set_bit(7) == 2 ** 7

The function `to_board(state)` takes an integer `state` as its argument that represents a state of the game tic-tac-toe.  It converts this state into a string that represents the board corresponding to the state.

In [ ]:
def to_board(state):
    result = '+-+-+-+\n'
    for cell in range(9):
        if state & (2 ** cell) != 0:
            result += '|X'
        elif state & (2 ** (cell + 9)) != 0:
            result += '|O'
        else:
            result += '| '
        if (cell + 1) % 3 == 0:
            result += '|\n+-+-+-+\n'
    return result

In [ ]:
print(to_board(set_bits([0,2,3,5,7,10,13,15])))

Given a `state` that is represented as a number, the function `empty(state)` returns the set of indexes of those cells such that neither player `X` nor player `O` has placed a mark in the cell.  Note that there are 9 cells on the board.  Each of these cells can hold either an `'X'` or an `'O'`.  If the $i^\textrm{th}$ cell is marked with a `'X'`, then the $i^\textrm{th}$ bit of `state` is set.  If instead the $i^\textrm{th}$ cell is marked with an `'O'`, then the $(9+i)^\textrm{th}$ bit of `state` is set.  If the $i^\textrm{th}$ cell is not yet marked, then both the $i^\textrm{th}$ bit and the $(9+i)^\textrm{th}$ are $0$.

In [ ]:
def empty(state):
    Free  = { n for n in range(9) }
    Free -= { n for n in range(9) if state & (1 << n) != 0 }
    Free -= { n for n in range(9) if state & (1 << (9 + n)) != 0 }
    return Free

In [ ]:
state = set_bits([2, 3, 5, 9+1, 9+4, 9+8])
print(to_board(state))
empty(state)

Given a `state` and the `player` who is next to move, the function `next_states(state, player)` computes the set of states that can be reached from `state`.
Note that player `X` is encoded as the number $0$, while player `O` is encoded as the number $1$.

In [ ]:
def next_states(state, player):
    Empty  = empty(state)
    Result = []
    for n in Empty:
        next_state = state | set_bit(player * 9 + n)
        Result.append(next_state)
    return Result

In [ ]:
state = set_bits([2,3,5,10,13,15])
print(f'state:\n{to_board(state)}')
print('next states:')                  
for s in next_states(state, 0):
    print(to_board(s))

The global variable `All_Lines` is a list of eight bit masks.  These masks can be used to test whether there are three identical marks in a row, column, or diagonal.

In [ ]:
All_Lines = [ set_bits([0,1,2]), # 1st row
              set_bits([3,4,5]), # 2nd row
              set_bits([6,7,8]), # 3rd row
              set_bits([0,3,6]), # 1st column
              set_bits([1,4,7]), # 2nd column
              set_bits([2,5,8]), # 3rd column
              set_bits([0,4,8]), # falling diagonal
              set_bits([2,4,6]), # rising diagonal
            ]

In [ ]:
for state in All_Lines:
    print(to_board(state))

The function `utility` takes two arguments:
- `state`  is an integer representing the board.
- `player` is a player, player `'X'`  is coded as the number `0`, player `'O'` is coded as `1`.
 
The function returns `1` if `player` has won the game, `-1` if the game is lost for `player`, `0` if its a draw, and `None` if the game has not yet been decided.

In [ ]:
def utility(state, player):
    for mask in All_Lines:
        if state & mask == mask:
            return 1 - 2 * player  # player 'X' has won
        if (state >> 9) & mask == mask:
            return -1 + 2 * player # player 'O' has won
    # 511 == 2**9 - 1 = 0b1_1111_1111  
    if (state & 511) | (state >> 9) != 511: # the board is not yet filled
        return None
    # at this point, the board has been filled, but there is no winner hence its a draw
    return 0 # it's a draw

In [ ]:
s1 = set_bits([0, 2, 3, 6, 1+9,  4+9, 5+9]) # 'X' has won
print(to_board(s1))
utility(s1, 0)

In [ ]:
s2 = set_bits([0, 2, 6, 8, 1+9, 4+9, 7+9]) # 'O' has won
print(to_board(s2))
utility(s2, 0)

In [ ]:
s3 = set_bits([0, 2, 5, 6, 7, 1+9, 3+9, 4+9, 8+9]) # it's a draw
print(to_board(s3))
print(utility(s3, 0))
s3

In [ ]:
s4 = set_bits([0, 2, 5, 6, 1+9, 3+9, 4+9]) # it ain't over yet
print(to_board(s4))
print(utility(s4, 0))

The function heuristic tries to guess the value of a state.  As it is never called in terminal states, it assumes that the game will be drawn.

In [ ]:
def heuristic(state, player):
    return 0

`finished(state)` is `True` if the game is over.

In [ ]:
finished = lambda state: utility(state, 0) != None

In [ ]:
s = set_bits([0, 2, 5, 6, 7, 1+9, 3+9, 4+9, 8+9])
print(to_board(s))
finished(s)

The function `get_move` asks the user to input a move in the format `r,c` where `r` is the row and the `c` is the column where the next symbol is to be placed.

In [ ]:
def get_move(state):
    while True:
        row, col = input('Enter move here: ').split(',')
        row, col = int(row), int(col)
        mask = set_bit(9 + row * 3 + col)
        if state & mask == 0:
            return state | mask
        else:
            print("Illegal input.  Please try again.")           

The function `final_msg(state)` informs the user about the result of the game once the game is finished.

In [ ]:
def final_msg(state):
    if finished(state):
        if utility(state, 1) == 1:
            print('You have won!')
        elif utility(state, 1) == -1:
            print('You have lost!')
        else:
            print("It's a draw.");
        return True
    return False

# Drawing the Board

In [ ]:
import ipycanvas as cnv

In [ ]:
size = 150

This function creates the canvas for the start state.  It draws an empty board which is later used for the game.

In [ ]:
def create_canvas():
    canvas = cnv.Canvas(size=(size * 3, size * 3 + 50))
    display(canvas)
    return canvas

The function `get_symbol(state, row, col)` takes three arguments:
* `state` is an integer representing a tic-tac-toe board.
* `row` is an integer in `range(3)` that specifies a row in the board.
* `col` is an integer in `range(3)` that specifies a column in the board.

The function returns the mark that is placed in the given `row`and `column`.  If this position is empty, a blank `' '` is returned. 

In [ ]:
def get_symbol(state, row, col):
    mask = set_bits([row * 3 + col])
    if mask & state == mask:
        return 'X'
    if mask & (state >> 9) == mask:
        return 'O'
    return ' '  

This function takes three arguments:
- `state` is the current state of the game.
- `canvas` is a canvas used to draw the state.
- `value` is the value of the game for player `X`.

The function draws the given `State` onto `canvas`.  Below that, the `value` is printed.

In [ ]:
def draw(state, canvas, value):
    canvas.clear()
    n = 3
    canvas.font          = '90px sans-serif'
    canvas.text_align    = 'center'
    canvas.text_baseline = 'middle'
    for row in range(n):
        for col in range(n):
            x = col * size
            y = row * size
            canvas.line_width = 3.0
            canvas.stroke_rect(x, y, size, size)
            symbol = get_symbol(state, row, col)
            if symbol != ' ':
                x += size // 2
                y += size // 2
                if symbol == 'X':
                    canvas.fill_style ='red'
                else:
                    canvas.fill_style ='blue'
                canvas.fill_text(symbol, x, y)
    canvas.font = '12px sans-serif'
    canvas.fill_style = 'green'
    for row in range(n):
        for col in range(n):
            x = col * size + 16
            y = row * size + 141
            canvas.fill_text(f'({row}, {col})', x, y)
    canvas.font = '20px sans-serif'
    canvas.fill_style = 'black'
    x = 1.5 * size
    y = 3.2 * size
    canvas.fill_text(str(value), x, y)

In [ ]:
draw(set_bits([0, 2, 5, 6, 1+9, 3+9, 4+9]), create_canvas(), -1)